[View in Colaboratory](https://colab.research.google.com/github/pirsquared/Colaboratory/blob/master/pandas_map.ipynb)

# Setup

In [0]:
import pandas as pd

df1 = pd.DataFrame(dict(
    sedol=[*'ABCDEF'],
    name='ZY YX XW WV VU UT'.split(),
    country=[f"C_{i}" for i in range(6)],
    sectorn=[45., 15., 55., 20., 30., 60.],
), pd.Index(['U_1', 'I_2', 'U_3', 'I_4', 'U_5', 'I_6']))

df2 = pd.Series({
    10: 'Energy',
    15: 'Materials',
    20: 'Industrials',
    25: 'Consumer Discretionary',
    30: 'Consumer Staples',
    35: 'Health Care',
    40: 'Financials',
    45: 'Information Technology',
    50: 'Telecommunication Services',
    55: 'Utilities',
    60: 'Real Estate'
}, name='sectorn_n').rename_axis('sectorn').to_frame()

## `map`
I suspect the problem was that you were attempting to map with a `DataFrame`.  In that [link](https://stackoverflow.com/questions/46789098/create-new-column-in-dataframe-with-match-values-from-other-dataframe) you gave, @jezrael is using a `pandas.Series` in the `map` and that makes all the difference.

You can pass one of two things to the `map` method

1. A callable object.  Most common callable object is a function.  Each element in the `pandas.Series` is passed to the callable and in its place is what is returned from the callable.
2. An object with a `get` method.  The `get` method would be used in place of the `callable`.  Both dictionaries and `pandas.Series` have a `get` method.


Ok, so what about your problem?  I suspect you tried this:

In [6]:
df1.sectorn.map(df2)

TypeError: ignored

When you wanted this:

In [7]:
df1.sectorn.map(df2.sectorn_n)

U_1    Information Technology
I_2                 Materials
U_3                 Utilities
I_4               Industrials
U_5          Consumer Staples
I_6               Real Estate
Name: sectorn, dtype: object

The subtle change from `df2` which is a `pandas.DataFrame` to `df2.sectorn_n` which is one column in the `pandas.DataFrame` and is also a `pandas.Series`.  Now, `df2.sectorn_n` has a `get` method and then magic happens.

This will assign that new column to the existing `df1`
```python
df1['sectorn_n'] = df1.sectorn.map(df2.sectorn_n)
```
But I use the `assign` method to create copies for demonstration so as not to alter the `pandas.DataFrame` I'm messing with.

In [12]:
df1.assign(sectorn_n=df1.sectorn.map(df2.sectorn_n))

,country,name,sectorn,sedol,sectorn_n
U_1,C_0,ZY,45.0,A,Information Technology
I_2,C_1,YX,15.0,B,Materials
U_3,C_2,XW,55.0,C,Utilities
I_4,C_3,WV,20.0,D,Industrials
U_5,C_4,VU,30.0,E,Consumer Staples
I_6,C_5,UT,60.0,F,Real Estate


## `join`
Another option.  Also produces a copy while leaving the original alone.

In [13]:
df1.join(df2, on='sectorn')

,country,name,sectorn,sedol,sectorn_n
U_1,C_0,ZY,45,A,Information Technology
I_2,C_1,YX,15,B,Materials
U_3,C_2,XW,55,C,Utilities
I_4,C_3,WV,20,D,Industrials
U_5,C_4,VU,30,E,Consumer Staples
I_6,C_5,UT,60,F,Real Estate


## `merge`

In [16]:
df1.merge(df2, left_on='sectorn', right_index=True)

,country,name,sectorn,sedol,sectorn_n
U_1,C_0,ZY,45,A,Information Technology
I_2,C_1,YX,15,B,Materials
U_3,C_2,XW,55,C,Utilities
I_4,C_3,WV,20,D,Industrials
U_5,C_4,VU,30,E,Consumer Staples
I_6,C_5,UT,60,F,Real Estate


In [17]:
df1.merge(df2.reset_index())

,country,name,sectorn,sedol,sectorn_n
0,C_0,ZY,45,A,Information Technology
1,C_1,YX,15,B,Materials
2,C_2,XW,55,C,Utilities
3,C_3,WV,20,D,Industrials
4,C_4,VU,30,E,Consumer Staples
5,C_5,UT,60,F,Real Estate
